Writing Functions That Accept Any Number of
Arguments

In [ ]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

In [ ]:
avg(1, 2) # 1.5
avg(1, 2, 3, 4) # 2.5

In [ ]:
import html
def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                        name=name,
                        attrs=attr_str,
                        value=html.escape(value))
    return element
# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)
# Creates '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

In [ ]:
def anyargs(*args, **kwargs):
    print(args) # A tuple
    print(kwargs) # A dict

In [ ]:
def a(x, *args, y):
    pass
def b(x, *args, y, **kwargs):
    pass

Writing Functions That Only Accept Keyword
Arguments

In [ ]:
def recv(maxsize, *, block):
    print('Receives a message')
    pass
#recv(1024, True) # TypeError
recv(1024, block=True) # Ok

In [ ]:
def mininum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

mininum(1, 5, 2, -5, 10) # Returns -5
mininum(1, 5, 2, -5, 10, clip=0) # Returns 0

Attaching Informational Metadata to Function
Arguments

In [ ]:
def add(x:int, y:int) -> int:
    return x + y

In [ ]:
help(add)

Returning Multiple Values from a Function

In [ ]:
def myfunc():
    return 1, 2, 3

In [ ]:
a,b,c = myfunc()

Defining Functions with Default Arguments

In [ ]:
def spam(a, b=42):
    print(a, b)
spam(1) # Ok. a=1, b=42
spam(1, 2) # Ok. a=1, b=2

In [ ]:
x = 42
def spam(a, b=x):
    print(a, b)

spam(1)

In [ ]:
x = 44

In [ ]:
def spam(a, b=[]):
    print(b)
    return b

x = spam(1)

In [ ]:
x

In [ ]:
x.append(99)
x.append('Yow!')

In [ ]:
x

In [ ]:
spam(1)

In [ ]:
spam(1) # OK
x = []
spam(1, x) # Silent error. x value overwritten by default
spam(1, 0) # Silent error. 0 ignored
spam(1, '') # Silent error. '' ignored

Defining Anonymous or Inline Functions

In [ ]:
add = lambda x, y, z: x + y + z
add(2,3,4)

In [ ]:
add('hello', 'world', 'siva')

In [ ]:
names = ['David Beazley', 'Brian Jones','Raymond Hettinger', 'Ned Batchelder']
sorted(names, key=lambda name: name.split()[-1].lower())

Capturing Variables in Anonymous Functions

In [ ]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y

In [ ]:
a(10)

In [ ]:
b(10)

In [ ]:
x = 15

In [ ]:
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))

In [ ]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

Making an N-Argument Callable Work As a Callable
with Fewer Arguments

In [ ]:
def spam(a, b, c, d):
    print(a, b, c, d)

In [ ]:
from functools import partial
s1 = partial(spam, 1) # a = 1
s1(2, 3, 4)

In [ ]:
s1(4, 5, 6)

In [ ]:
s2 = partial(spam, d=42) # d = 42
s2(1, 2, 3)

In [ ]:
s3 = partial(spam, 1, 2, d=42) # a = 1, b = 2, d = 42
s3(3)

In [ ]:
s3(4)

In [ ]:
s3(5)

In [ ]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]
import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

In [ ]:
pt = (4, 3)
points.sort(key=partial(distance,pt))
points

In [ ]:
from functools import partial
serv = TCPServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))
serv.serve_forever()

Replacing Single Method Classes with Functions

In [ ]:
from urllib.request import urlopen
class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))
# Example use. Download stock data from yahoo
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

In [ ]:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

Carrying Extra State with Callback Functions

In [ ]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

In [ ]:
def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)

In [ ]:
apply_async(add, ('hello', 'world'), callback=print_result)

Inlining Callback Functions

In [9]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

In [6]:
from queue import Queue
from functools import wraps
class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args
        
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

In [10]:
def add(x, y):
    return x + y
@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [11]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


In [ ]:
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
    apply_async = pool.apply_async
    # Run the test function
    test()

Accessing Variables Defined Inside a Closure

In [ ]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)
    # Accessor methods for n
    def get_n():
        return n
    def set_n(value):
        nonlocal n
        n = value
    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

In [ ]:
f = sample()

In [ ]:
f()

In [ ]:
f.set_n(10)
f()
f.get_n()